In [1]:
# importing necessary libraries
import scipy.io
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from scipy.signal import hilbert
from pyhht.visualization import plot_imfs
from numpy import asarray
from tqdm import tqdm
import re
from PIL import Image
import time
import pywt
import pyhht
from pyhht import EMD

In [2]:
data_names = []
for i in range(1, 20):
    my_string = 's'+str(i)+'D.mat'
    data_names.append(my_string)

In [3]:
# specified data and label directory
data_dir = '/home/kiara/Desktop/EEG_Project1/dis_data/'
label_dir = '/home/kiara/Desktop/EEG_Project1/dis_label/'
# specifiying file names for label
label_names = data_names # data and label has the same name

In [4]:
# list of training data
print("Training Data is:")
print(data_names)
print("\nTraining Label is:")
print(label_names)

Training Data is:
['s1D.mat', 's2D.mat', 's3D.mat', 's4D.mat', 's5D.mat', 's6D.mat', 's7D.mat', 's8D.mat', 's9D.mat', 's10D.mat', 's11D.mat', 's12D.mat', 's13D.mat', 's14D.mat', 's15D.mat', 's16D.mat', 's17D.mat', 's18D.mat', 's19D.mat']

Training Label is:
['s1D.mat', 's2D.mat', 's3D.mat', 's4D.mat', 's5D.mat', 's6D.mat', 's7D.mat', 's8D.mat', 's9D.mat', 's10D.mat', 's11D.mat', 's12D.mat', 's13D.mat', 's14D.mat', 's15D.mat', 's16D.mat', 's17D.mat', 's18D.mat', 's19D.mat']


In [5]:
# defining variables
# choosing a file for data extraction
file_name1 = data_names[18]
print(file_name1)

s19D.mat


In [6]:
# extracting the data
load_file1 = os.path.join(data_dir, file_name1)
data1 = scipy.io.loadmat(load_file1)
data = data1['DATA']
data = data.T

In [7]:
# extracting the label
load_file2 = os.path.join(label_dir, file_name1)
data2 = scipy.io.loadmat(load_file2)
label = data2['dist_label']

In [8]:
# generating a random number
m = re.search('s(.+?)D', file_name1)
random_state_value = int(m.group(1))

In [9]:
# initializing log directory and model name
m = re.search('s(.+?)D', file_name1)
log_directory = 'Log_hht_p'+str(m.group(1))
model_name = 'model_hht_p'+str(m.group(1))+'.h5'

In [10]:
# reshaping training data
print(data.shape)
d = np.reshape(data, (data.shape[0], data.shape[2], data.shape[1]))
print(d.shape)

(48, 2828, 64)
(48, 64, 2828)


In [11]:
start_time =  time.time()
no_of_imfs = 1                                              # 1, 2, 3, 4
x_t = np.zeros((d.shape[0], d.shape[1], no_of_imfs, d.shape[2]), dtype = np.float32)
for participant in range(0, d.shape[0]):
    for value in range(0, d.shape[1]):
        decomposer = EMD(d[participant][value])
        # decomposing the input signal into IMFs
        imfs = decomposer.decompose()
        x_t[participant][value][0] = imfs[1]
        #x_t[participant][value][1] = imfs[2]
        #x_t[participant][value][2] = imfs[3]
        #x_t[participant][value][3] = imfs[4]

Mode 8, Iteration 600
0 minima > 0; 8 maxima < 0.
Mode 9, Iteration 600
1 minima > 0; 4 maxima < 0.
Mode 8, Iteration 600
0 minima > 0; 8 maxima < 0.
Mode 8, Iteration 800
0 minima > 0; 8 maxima < 0.
Mode 8, Iteration 1000
0 minima > 0; 8 maxima < 0.
Mode 9, Iteration 600
0 minima > 0; 3 maxima < 0.
Mode 8, Iteration 600
0 minima > 0; 11 maxima < 0.
Mode 8, Iteration 600
0 minima > 0; 6 maxima < 0.
Mode 8, Iteration 800
0 minima > 0; 6 maxima < 0.
Mode 8, Iteration 1000
0 minima > 0; 6 maxima < 0.
Mode 8, Iteration 1200
0 minima > 0; 6 maxima < 0.
Mode 9, Iteration 1400
0 minima > 0; 4 maxima < 0.
Mode 9, Iteration 600
0 minima > 0; 5 maxima < 0.
Mode 9, Iteration 600
0 minima > 0; 4 maxima < 0.
Mode 9, Iteration 800
0 minima > 0; 4 maxima < 0.
Mode 9, Iteration 1000
0 minima > 0; 4 maxima < 0.
Mode 8, Iteration 600
0 minima > 0; 5 maxima < 0.
Mode 9, Iteration 600
0 minima > 0; 5 maxima < 0.
Mode 7, Iteration 600
0 minima > 0; 14 maxima < 0.
Mode 7, Iteration 800
0 minima > 0; 14 maxi

In [12]:
# obtaining real and imaginary parts of the hht transformed signal
signal = hilbert(x_t)
signal_real = np.real(signal)
signal_imag = np.imag(signal)

# calculating amplitude, instantaneous frequency and instantaneous phase
from scipy import angle, unwrap
inst_phase = np.unwrap(np.angle(signal)) # unwrapped instantaneous phase
inst_freq = np.diff(inst_phase) # instantaneous frequency
amplitude = np.abs(signal)

# calculating frequency
freq = np.zeros((amplitude.shape[0], amplitude.shape[1], amplitude.shape[2], amplitude.shape[3]), dtype = np.float32)
sh = inst_freq.shape[3] - 1
for i in range(amplitude.shape[0]):
    for j in range(amplitude.shape[1]):
        for k in range(amplitude.shape[2]):
            freq[i][j][k] = np.append(inst_freq[i][j][k], inst_freq[i][j][k][sh])  
            
#xt = np.concatenate((amplitude, freq), axis=1)
xt = np.stack((amplitude, freq), axis=1)
# reshaping the data
xt = xt.reshape(xt.shape[0], xt.shape[1] * xt.shape[2] * xt.shape[3], xt.shape[4])

In [ ]:
'''
xt = np.dstack((signal_real, signal_imag))
xt = xt.reshape(xt.shape[0], xt.shape[1] * xt.shape[2], xt.shape[3])
'''

In [13]:
# Performing train - test split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(xt, label, test_size=0.2, random_state = random_state_value)

In [14]:
print(X_Train.shape)
print(X_Test.shape)
print(Y_Train.shape)
print(Y_Test.shape)

(38, 128, 2828)
(10, 128, 2828)
(38, 1)
(10, 1)


In [15]:
# Defining the model
modeltwo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_Train.shape[1], X_Train.shape[2])),
    tf.keras.layers.Dense(64, activation=tf.nn.tanh),
    tf.keras.layers.Dense(64,activation=tf.nn.tanh),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
modeltwo.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics='acc')
modeltwo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 361984)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                23167040  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 23,171,265
Trainable params: 23,171,265
Non-trainable params: 0
_________________________________________________________________


In [16]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 500, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = log_directory)
]

In [17]:
# model training
for i in range(0, 4):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(xt, label, test_size=0.2, random_state = random_state_value+i)
    results = modeltwo.fit(X_Train, Y_Train, validation_split = 0.2, batch_size = 4, epochs = 500, verbose=1, callbacks=callbacks)
modeltwo.save(model_name)

Epoch 1/500
8/8 [==============================] - 1s 62ms/step - loss: 1.4277 - acc: 0.4333 - val_loss: 1.3630 - val_acc: 0.2500
Epoch 2/500
8/8 [==============================] - 0s 10ms/step - loss: 0.6647 - acc: 0.6667 - val_loss: 0.5200 - val_acc: 0.8750
Epoch 3/500
8/8 [==============================] - 0s 11ms/step - loss: 0.5261 - acc: 0.7333 - val_loss: 0.7939 - val_acc: 0.5000
Epoch 4/500
8/8 [==============================] - 0s 11ms/step - loss: 0.3710 - acc: 0.8667 - val_loss: 0.6715 - val_acc: 0.6250
Epoch 5/500
8/8 [==============================] - 0s 11ms/step - loss: 0.2574 - acc: 0.9000 - val_loss: 0.3469 - val_acc: 0.8750
Epoch 6/500
8/8 [==============================] - 0s 11ms/step - loss: 0.2115 - acc: 0.9667 - val_loss: 0.9108 - val_acc: 0.6250
Epoch 7/500
8/8 [==============================] - 0s 11ms/step - loss: 0.1409 - acc: 0.9667 - val_loss: 0.4727 - val_acc: 0.7500
Epoch 8/500
8/8 [==============================] - 0s 11ms/step - loss: 0.1403 - acc: 0.96

8/8 [==============================] - 0s 11ms/step - loss: 9.2563e-09 - acc: 1.0000 - val_loss: 2.3254 - val_acc: 0.7500
Epoch 63/500
8/8 [==============================] - 0s 11ms/step - loss: 9.2440e-09 - acc: 1.0000 - val_loss: 2.3251 - val_acc: 0.7500
Epoch 64/500
8/8 [==============================] - 0s 11ms/step - loss: 8.6058e-09 - acc: 1.0000 - val_loss: 2.3241 - val_acc: 0.7500
Epoch 65/500
8/8 [==============================] - 0s 11ms/step - loss: 7.8773e-09 - acc: 1.0000 - val_loss: 2.3403 - val_acc: 0.7500
Epoch 66/500
8/8 [==============================] - 0s 11ms/step - loss: 7.6605e-09 - acc: 1.0000 - val_loss: 2.3537 - val_acc: 0.7500
Epoch 67/500
8/8 [==============================] - 0s 10ms/step - loss: 7.4626e-09 - acc: 1.0000 - val_loss: 2.3663 - val_acc: 0.7500
Epoch 68/500
8/8 [==============================] - 0s 10ms/step - loss: 7.2326e-09 - acc: 1.0000 - val_loss: 2.3775 - val_acc: 0.7500
Epoch 69/500
8/8 [==============================] - 0s 10ms/step - l

8/8 [==============================] - 0s 10ms/step - loss: 3.9395e-09 - acc: 1.0000 - val_loss: 2.5740 - val_acc: 0.7500
Epoch 123/500
8/8 [==============================] - 0s 11ms/step - loss: 3.9332e-09 - acc: 1.0000 - val_loss: 2.5767 - val_acc: 0.7500
Epoch 124/500
8/8 [==============================] - 0s 11ms/step - loss: 3.9168e-09 - acc: 1.0000 - val_loss: 2.5793 - val_acc: 0.7500
Epoch 125/500
8/8 [==============================] - 0s 11ms/step - loss: 3.9147e-09 - acc: 1.0000 - val_loss: 2.5819 - val_acc: 0.7500
Epoch 126/500
8/8 [==============================] - 0s 11ms/step - loss: 3.9212e-09 - acc: 1.0000 - val_loss: 2.5843 - val_acc: 0.7500
Epoch 127/500
8/8 [==============================] - 0s 11ms/step - loss: 3.9236e-09 - acc: 1.0000 - val_loss: 2.5867 - val_acc: 0.7500
Epoch 128/500
8/8 [==============================] - 0s 10ms/step - loss: 3.9059e-09 - acc: 1.0000 - val_loss: 2.5901 - val_acc: 0.7500
Epoch 129/500
8/8 [==============================] - 0s 11ms/s

8/8 [==============================] - 0s 11ms/step - loss: 3.5385e-09 - acc: 1.0000 - val_loss: 2.6663 - val_acc: 0.7500
Epoch 183/500
8/8 [==============================] - 0s 11ms/step - loss: 3.5534e-09 - acc: 1.0000 - val_loss: 2.6675 - val_acc: 0.7500
Epoch 184/500
8/8 [==============================] - 0s 11ms/step - loss: 3.5469e-09 - acc: 1.0000 - val_loss: 2.6688 - val_acc: 0.7500
Epoch 185/500
8/8 [==============================] - 0s 11ms/step - loss: 3.5537e-09 - acc: 1.0000 - val_loss: 2.6700 - val_acc: 0.7500
Epoch 186/500
8/8 [==============================] - 0s 11ms/step - loss: 3.5511e-09 - acc: 1.0000 - val_loss: 2.6712 - val_acc: 0.7500
Epoch 187/500
8/8 [==============================] - 0s 11ms/step - loss: 3.5462e-09 - acc: 1.0000 - val_loss: 2.6725 - val_acc: 0.7500
Epoch 188/500
8/8 [==============================] - 0s 10ms/step - loss: 3.5510e-09 - acc: 1.0000 - val_loss: 2.6737 - val_acc: 0.7500
Epoch 189/500
8/8 [==============================] - 0s 10ms/s

8/8 [==============================] - 0s 11ms/step - loss: 3.2167e-09 - acc: 1.0000 - val_loss: 2.7097 - val_acc: 0.7500
Epoch 243/500
8/8 [==============================] - 0s 10ms/step - loss: 3.2192e-09 - acc: 1.0000 - val_loss: 2.7106 - val_acc: 0.7500
Epoch 244/500
8/8 [==============================] - 0s 10ms/step - loss: 3.2146e-09 - acc: 1.0000 - val_loss: 2.7116 - val_acc: 0.7500
Epoch 245/500
8/8 [==============================] - 0s 10ms/step - loss: 3.2224e-09 - acc: 1.0000 - val_loss: 2.7125 - val_acc: 0.7500
Epoch 246/500
8/8 [==============================] - 0s 11ms/step - loss: 3.2242e-09 - acc: 1.0000 - val_loss: 2.7134 - val_acc: 0.7500
Epoch 247/500
8/8 [==============================] - 0s 11ms/step - loss: 3.2256e-09 - acc: 1.0000 - val_loss: 2.7143 - val_acc: 0.7500
Epoch 248/500
8/8 [==============================] - 0s 11ms/step - loss: 3.2264e-09 - acc: 1.0000 - val_loss: 2.7151 - val_acc: 0.7500
Epoch 249/500
8/8 [==============================] - 0s 11ms/s

...

8/8 [==============================] - 0s 10ms/step - loss: 1.3399e-09 - acc: 1.0000 - val_loss: 1.1563 - val_acc: 0.8750
Epoch 232/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3427e-09 - acc: 1.0000 - val_loss: 1.1571 - val_acc: 0.8750
Epoch 233/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3464e-09 - acc: 1.0000 - val_loss: 1.1579 - val_acc: 0.8750
Epoch 234/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3475e-09 - acc: 1.0000 - val_loss: 1.1588 - val_acc: 0.8750
Epoch 235/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3481e-09 - acc: 1.0000 - val_loss: 1.1597 - val_acc: 0.8750
Epoch 236/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3529e-09 - acc: 1.0000 - val_loss: 1.1605 - val_acc: 0.8750
Epoch 237/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3543e-09 - acc: 1.0000 - val_loss: 1.1616 - val_acc: 0.8750
Epoch 238/500
8/8 [==============================] - 0s 10ms/s

8/8 [==============================] - 0s 11ms/step - loss: 1.2321e-09 - acc: 1.0000 - val_loss: 1.1400 - val_acc: 0.8750
Epoch 292/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2338e-09 - acc: 1.0000 - val_loss: 1.1408 - val_acc: 0.8750
Epoch 293/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2337e-09 - acc: 1.0000 - val_loss: 1.1416 - val_acc: 0.8750
Epoch 294/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2370e-09 - acc: 1.0000 - val_loss: 1.1424 - val_acc: 0.8750
Epoch 295/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2407e-09 - acc: 1.0000 - val_loss: 1.1432 - val_acc: 0.8750
Epoch 296/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2423e-09 - acc: 1.0000 - val_loss: 1.1440 - val_acc: 0.8750
Epoch 297/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2427e-09 - acc: 1.0000 - val_loss: 1.1452 - val_acc: 0.8750
Epoch 298/500
8/8 [==============================] - 0s 11ms/s

8/8 [==============================] - 0s 11ms/step - loss: 1.1108e-09 - acc: 1.0000 - val_loss: 1.1037 - val_acc: 0.8750
Epoch 352/500
8/8 [==============================] - 0s 11ms/step - loss: 1.1112e-09 - acc: 1.0000 - val_loss: 1.1052 - val_acc: 0.8750
Epoch 353/500
8/8 [==============================] - 0s 10ms/step - loss: 1.1164e-09 - acc: 1.0000 - val_loss: 1.1062 - val_acc: 0.8750
Epoch 354/500
8/8 [==============================] - 0s 10ms/step - loss: 1.1150e-09 - acc: 1.0000 - val_loss: 1.1072 - val_acc: 0.8750
Epoch 355/500
8/8 [==============================] - 0s 10ms/step - loss: 1.1188e-09 - acc: 1.0000 - val_loss: 1.1082 - val_acc: 0.8750
Epoch 356/500
8/8 [==============================] - 0s 10ms/step - loss: 1.1180e-09 - acc: 1.0000 - val_loss: 1.1091 - val_acc: 0.8750
Epoch 357/500
8/8 [==============================] - 0s 10ms/step - loss: 1.1208e-09 - acc: 1.0000 - val_loss: 1.1103 - val_acc: 0.8750
Epoch 358/500
8/8 [==============================] - 0s 10ms/s

8/8 [==============================] - 0s 10ms/step - loss: 1.2250e-09 - acc: 1.0000 - val_loss: 1.1573 - val_acc: 0.8750
Epoch 412/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2243e-09 - acc: 1.0000 - val_loss: 1.1582 - val_acc: 0.8750
Epoch 413/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2262e-09 - acc: 1.0000 - val_loss: 1.1588 - val_acc: 0.8750
Epoch 414/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2285e-09 - acc: 1.0000 - val_loss: 1.1593 - val_acc: 0.8750
Epoch 415/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2316e-09 - acc: 1.0000 - val_loss: 1.1599 - val_acc: 0.8750
Epoch 416/500
8/8 [==============================] - 0s 10ms/step - loss: 1.2327e-09 - acc: 1.0000 - val_loss: 1.1606 - val_acc: 0.8750
Epoch 417/500
8/8 [==============================] - 0s 11ms/step - loss: 1.2341e-09 - acc: 1.0000 - val_loss: 1.1612 - val_acc: 0.8750
Epoch 418/500
8/8 [==============================] - 0s 11ms/s

8/8 [==============================] - 0s 10ms/step - loss: 1.3289e-09 - acc: 1.0000 - val_loss: 1.1915 - val_acc: 0.8750
Epoch 472/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3328e-09 - acc: 1.0000 - val_loss: 1.1920 - val_acc: 0.8750
Epoch 473/500
8/8 [==============================] - 0s 11ms/step - loss: 1.3326e-09 - acc: 1.0000 - val_loss: 1.1925 - val_acc: 0.8750
Epoch 474/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3354e-09 - acc: 1.0000 - val_loss: 1.1932 - val_acc: 0.8750
Epoch 475/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3372e-09 - acc: 1.0000 - val_loss: 1.1936 - val_acc: 0.8750
Epoch 476/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3382e-09 - acc: 1.0000 - val_loss: 1.1943 - val_acc: 0.8750
Epoch 477/500
8/8 [==============================] - 0s 10ms/step - loss: 1.3405e-09 - acc: 1.0000 - val_loss: 1.1947 - val_acc: 0.8750
Epoch 478/500
8/8 [==============================] - 0s 10ms/s

In [18]:
# model evaluation on test data
h_scores = modeltwo.evaluate(X_Test, Y_Test, verbose=1)

1/1 [==============================] - 0s 80ms/step - loss: 4.6176e-04 - acc: 1.0000


In [19]:
# predicting test data values
Y_hat = modeltwo.predict(X_Test)
Y_pred = []
for item in Y_hat:
    if item>=(np.mean(Y_hat)):
        Y_pred.append(1)
    else:
        Y_pred.append(0)

correct = 0
for i in range(len(Y_pred)):
    if Y_Test[i] == Y_pred[i]:
        correct += 1
h_accuracy = correct/len(Y_pred)

In [20]:
# obtaining accuracy values
print("Accuracy from HHT")
print(h_accuracy)

Accuracy from HHT
1.0
